In [ ]:
!pip install langchain
!pip install PyPDF2
!pip install sqlalchemy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install langchain langchain-community PyPDF2 sqlalchemy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [5]:

import os
import requests
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from sqlalchemy import create_engine, Column, String, Float, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
# Upload PDF file
from google.colab import files
uploaded = files.upload()

# Get the filename of the uploaded PDF
pdf_file_path = list(uploaded.keys())[0]


Saving cricket.pdf to cricket.pdf


In [7]:
# Load PDF and create chunks
def load_pdf_and_split(file_path, chunk_size=1000):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=100)
    chunks = loader.load_and_split(text_splitter)
    return chunks

# Create embeddings
def create_embeddings():
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    print(embeddings)
    return embeddings

In [10]:
chunks=load_pdf_and_split(pdf_file_path)
embeddings=create_embeddings()

<ipython-input-7-e60ac264e717>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [ ]:
pip install langchain-google-genai pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.7 MB/s eta 0:00:00


In [11]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 12.8 MB/s eta 0:00:00


In [12]:
vector_store = FAISS.from_documents(chunks, embeddings)

In [13]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})


In [24]:
# Define a prompt template to guide AI responses with a specific structure
from google.colab import userdata
api_key = userdata.get('api_key')

from google.colab import userdata
userdata.get('api_key')
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
def promptTemplate(template_text):
    template = template_text
    prompt = PromptTemplate.from_template(template)

    # Create a document chain that combines retrieved chunks for more contextual responses
    combine_docs_chain = create_stuff_documents_chain(llm, prompt)

    # Create the final retrieval chain that integrates retrieval and response generation
    retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
    return retrieval_chain

text = '''You are a helpful AI assistant. Answer based on the context provided.
    If you don't know, say I don't know
    context: {context} input: {input} answer:'''

retrieval_chain = promptTemplate(text)

In [23]:
def getPromptsAndReturnResponse(prompt):
    response = retrieval_chain.invoke({"input": prompt})
    print(response["answer"])
getPromptsAndReturnResponse("what is batting")


Batting in cricket is probably the most popular of all skills. Firstly, it allows the player to score, and secondly he has the full attention of not only both teams, but also the spectators.


In [ ]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
chunks

[Document(metadata={'source': 'cricket (1).pdf', 'page': 0}, page_content='Cricket\nTHE ROYAL MARINES'),
 Document(metadata={'source': 'cricket (1).pdf', 'page': 1}, page_content='2\nStudent Notes                                      Teacher Notes\nCONTENTS\nPrepared for The Royal Navy in Association with The National Cricket\nAssociation and produced by Education & Youth Ltd, London.\nIf there is any support you feel The Royal Navy can give regarding this\nproject please contact 0870 333 0423.\nAcknowledgements\nThese notes have been compiled by Bob Carter – National Coach,\n(Coach Education) The National Cricket Association\nTechnical Editor: P Edwards M.A.\nMESSAGE FROM THE NATIONAL CRICKET ASSOCIATION 3\nINTRODUCTION TO THE MODULE 3\nSKILLS DEVELOPMENT SHEET 4\nUNIT 1 INTRODUCTION TO CRICKET MODULE 5\nUNIT 2 TECHNIQUES AND SKILLS 6\n1 FIELDING 6\n2 BOWLING 9\n3 BATTING 14\n4 WICKET KEEPING 22\nUNIT 3 TACTICS AND FIELD PLACEMENT 25\nUNIT 4 UMPIRING 28\nUNIT 5 PHYSICAL CONDITIONING F

In [ ]:
vector_store

In [ ]:
# Function to print the contents of the vector database
def print_vector_store_contents(retriever, query="*"):
    print("Vector Store Contents:")
    documents = retriever.get_relevant_documents(query)
    for i, doc in enumerate(documents):
        print(f"Document {i + 1}:")
        print(doc.page_content)
        print("\n")

# Create FAISS vector store from the chunks


# Set up the retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# Print the contents of the vector store
print_vector_store_contents(retriever)


Vector Store Contents:
Document 1:
Cricket
THE ROYAL MARINES


Document 2:
of physical activity involved. It is also possible to structure the skill
learning so that simple competitions and small sided games prevent
the practices from becoming boring. They also provide opportunities
of introducing to beginners, in stages, small sections of the 42 Laws
governing the playing of the adult game.
The material covered in this book does not relate to any specific
Examination Board’s syllabus, but is intended to provide a basic
resource which should be supported and supplemented by reference
to the texts listed in Recommended Reading.


Document 3:
29
Table 1:  The Umpire Signals
No-Ball Wide Leg-bye
Bye Six Four. Arm waved.
Dead Ball also called Out Short Run. The number of runs short to be called.
NB the umpire will not allow the game to continue until the scorers
have acknowledged his signal.


Document 4:
21
Figure 35
Figure 34
Making One’s Ground
Always run the first run as fast as
possib

<ipython-input-24-fe7fd005c0da>:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  documents = retriever.get_relevant_documents(query)


In [ ]:
import numpy as np

# Function to print the embeddings stored in the vector store
def print_embeddings(vector_store):
    embeddings = vector_store.faiss_index.reconstruct_n(0, vector_store.faiss_index.ntotal)
    print("Embeddings stored in the vector store:")
    for i, embedding in enumerate(embeddings):
        print(f"Embedding {i + 1}: {embedding[:10]}...")  # Printing only the first 10 values of each embedding for brevity

# Create FAISS vector store from the chunks
vector_store = create_embeddings()

# Print the embeddings stored in the vector store
print_embeddings(vector_store)


client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'faiss_index'

AttributeError: 'FAISS' object has no attribute 'faiss_index'

In [ ]:
import numpy as np

# Function to print the embeddings stored in the vector store
def print_embeddings(vector_store):
    # Access the index from the FAISS vector store
    faiss_index = vector_store.index
    embeddings = faiss_index.reconstruct_n(0, faiss_index.ntotal)
    print("Embeddings stored in the vector store:")
    for i, embedding in enumerate(embeddings):
        print(f"Embedding {i + 1}: {embedding[:10]}...")  # Printing only the first 10 values of each embedding for brevity

# Function to create embeddings and FAISS vector store from chunks
def create_embeddings():
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(chunks, embeddings)
    return vector_store

# Create FAISS vector store from the chunks
vector_store = create_embeddings()

# Print the embeddings stored in the vector store
print_embeddings(vector_store)


Embeddings stored in the vector store:
Embedding 1: [-0.01111323  0.06118551 -0.02760149 -0.02080865  0.01520091  0.03927991
 -0.00861595 -0.05758078  0.00025856  0.09111562]...
Embedding 2: [-0.00143545  0.01505857 -0.01734007 -0.03409294  0.00356306  0.06062359
 -0.0534826   0.03346577 -0.07100087  0.1115301 ]...
Embedding 3: [ 0.02033223  0.07422196 -0.11318658 -0.10544559 -0.02572958  0.04824719
  0.00912656  0.05860418  0.02502922  0.10294083]...
Embedding 4: [ 0.0273876   0.0561967  -0.03795458 -0.06722938 -0.06603194  0.05635621
 -0.03003717  0.04332043 -0.0259237   0.05994638]...
Embedding 5: [-0.01056792  0.05386079 -0.07010289 -0.0080677   0.02000069  0.07566515
 -0.01463818  0.06414424  0.03106127  0.1305437 ]...
Embedding 6: [ 0.05948059  0.01051815 -0.04129196 -0.04995469 -0.06809552  0.02399878
 -0.02733152  0.03582982 -0.03863531  0.08915082]...
Embedding 7: [ 0.05513946  0.03696148 -0.05745225 -0.03353872 -0.02910757  0.06229207
 -0.01121592 -0.00772762 -0.04553648  0.0

In [2]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [4]:
pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.8 MB/s eta 0:00:00


In [9]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.4 MB/s eta 0:00:00
